In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os
from torchvision.utils import save_image

In [2]:
# Define Variational Autoencoder
class VAE(nn.Module):
    def __init__(self, latent_dim=128):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(3*64*64, 512),
            nn.ReLU(),
            nn.Linear(512, latent_dim*2))
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 3*64*64),
            nn.Sigmoid())

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        x = x.view(x.size(0), -1)
        h = self.encoder(x)
        mu, logvar = h.chunk(2, dim=-1)
        z = self.reparameterize(mu, logvar)
        x_hat = self.decoder(z)
        return x_hat, mu, logvar

def vae_loss(recon_x, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(recon_x, x.view(-1, 3*64*64), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [4]:
# Load data
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])
dataset = ImageFolder('/content/drive/MyDrive/Colab Notebooks/Mini Project/', transform=transform)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [7]:
# Train
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vae = VAE().to(device)
optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)

for epoch in range(10):  # Tune this
    for batch, _ in loader:
        batch = batch.to(device)
        recon_batch, mu, logvar = vae(batch)
        loss = vae_loss(recon_batch, batch, mu, logvar)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}], Loss: {loss.item():.2f}")
    save_image(batch[0], f"/content/drive/MyDrive/Colab Notebooks/Mini Project/original_epoch{epoch+1}.png")
    save_image(recon_batch[0].view(3, 64, 64), f"/content/drive/MyDrive/Colab Notebooks/Mini Project/recon_epoch{epoch+1}.png")
    torch.save(vae.state_dict(), "/content/drive/MyDrive/Colab Notebooks/Mini Project/vae_model.pth")

Epoch [1], Loss: 171992.81
Epoch [2], Loss: 186095.23
Epoch [3], Loss: 173264.36
Epoch [4], Loss: 172199.09
Epoch [5], Loss: 170922.94
Epoch [6], Loss: 170683.64
Epoch [7], Loss: 170690.97
Epoch [8], Loss: 170620.61
Epoch [9], Loss: 170579.23
Epoch [10], Loss: 170551.53
